# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import math
import xlsxwriter
import requests
import yfinance as yf
from concurrent.futures import ThreadPoolExecutor, as_completed

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
stocks = stocks[['Symbol', 'Security']]
stocks = stocks.rename(columns={'Symbol': 'Ticker', 'Security': 'Name'})
stocks

,Ticker,Name
0,MMM,3M
1,AOS,A. O. Smith
2,ABT,Abbott
3,ABBV,AbbVie
4,ACN,Accenture
...,...,...
498,XYL,Xylem Inc.
499,YUM,Yum! Brands
500,ZBRA,Zebra Technologies
501,ZBH,Zimmer Biomet


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from my_secrets import IEX_CLOUD_API_TOKEN
from my_secrets import ALPHA_VANTAGE_API_KEY

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [12]:
ticker = 'AAPL'
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
# url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={ALPHA_VANTAGE_API_KEY}'
# r = requests.get(url)
# data_cap = r.json()
# data_cap
# cap = data_cap['MarketCapitalization']

# url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&apikey={ALPHA_VANTAGE_API_KEY}'
# r = requests.get(url)
# data_close = r.json()

# close = data_close['Time Series (Daily)']
# close

# print(f'{ticker} has a market cap of {cap} and a closing price of {close}')

In [5]:
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL']

# Fetching data
# data = yf.download(tickers, period='1d')

# Displaying the closing prices and market cap
# for ticker in tickers:
#     stock = yf.Ticker(ticker)
#     close_price = data['Close'][ticker].iloc[0]
#     market_cap = stock.info['marketCap']
#     print(f"{ticker} - Close Price: {close_price}, Market Cap: {market_cap}")


data = yf.download(tickers, period='1d')
stock = yf.Ticker('AAPL')
stock
# dir(stock)

[*********************100%%**********************]  4 of 4 completed


yfinance.Ticker object <AAPL>

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [7]:
price = stock.history(period='1d')['Close'].iloc[0]
print('Close Price: $', price)
market_cap = stock.info['marketCap']
print('Market Cap: ', market_cap/10**9, 'Billion')

Close Price: $ 223.0498046875
Market Cap:  3420.117532672 Billion


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [8]:
my_columns = ['ticker', 'stock_price', 'market_cap', 'number_of_shares_to_buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,ticker,stock_price,market_cap,number_of_shares_to_buy


In [9]:
new_row = pd.Series(
    [
        'AAPL',
        price,
        market_cap,
        'N/A'
    ],
    index=my_columns
)

# Convert the Series to a DataFrame and concatenate it with the existing DataFrame
pd.concat([final_dataframe, new_row.to_frame().T], ignore_index=True)

,ticker,stock_price,market_cap,number_of_shares_to_buy
0,AAPL,223.049805,3420117532672,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [10]:
# final_dataframe = pd.DataFrame(columns=my_columns)
# for ticker in stocks['Ticker']:
#     try:
#         stock = yf.Ticker(ticker)
#         history = stock.history(period='1d')
        
#         if not history.empty:
#             price = history['Close'].iloc[0]
#             market_cap = stock.info['marketCap']
#         else:
#             price = None
#             market_cap = None

#         new_row = pd.Series(
#             [
#                 ticker,
#                 price,
#                 market_cap,
#                 'N/A'
#             ],
#             index=my_columns
#         )
        
#         # Append the new row to the DataFrame
#         final_dataframe = pd.concat([final_dataframe, new_row.to_frame().T], ignore_index=True)
#     except:
#         continue

In [11]:
# final_dataframe

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [12]:
final_dataframe = pd.DataFrame(columns=my_columns)
def fetch_stock_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        history = stock.history(period='1d')
        
        if not history.empty:
            price = history['Close'].iloc[0]
            market_cap = stock.info['marketCap']
        else:
            price = None
            market_cap = None
        
        return pd.Series(
            [
                ticker,
                price,
                market_cap,
                'N/A'
            ],
            index=my_columns
        )
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.Series([ticker, None, None, 'N/A'], index=my_columns)

# Using ThreadPoolExecutor to fetch data concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(fetch_stock_data, ticker) for ticker in stocks['Ticker']]
    
    for future in as_completed(futures):
        final_dataframe = pd.concat([final_dataframe, future.result().to_frame().T], ignore_index=True)

BRK.B: No data found, symbol may be delisted
BF.B: Period '1d' is invalid, must be one of ['1mo', '3mo', '6mo', 'ytd', '1y', '2y', '5y', '10y', 'max']


In [13]:
final_dataframe = final_dataframe[(final_dataframe['stock_price'].notna()) & (final_dataframe['market_cap'].notna())]


In [14]:
final_dataframe[final_dataframe['stock_price'].isnull()]


,ticker,stock_price,market_cap,number_of_shares_to_buy


In [15]:
final_dataframe.index = range(len(final_dataframe))

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [18]:
portfolio_size = input('Enter the value of your portfolio: ')

try:
    val = float(portfolio_size)
except ValueError:
    print('That is not a number. Please try again.')
    portfolio_size = input('Enter the value of your portfolio: ')
    val = float(portfolio_size)

print('Portfolio Size: ', "{:,}".format(val))

Portfolio Size:  1,000,000.0


In [19]:
position_size = val/len(final_dataframe.index)
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'number_of_shares_to_buy'] = math.floor(position_size/final_dataframe.loc[i, 'stock_price'])

final_dataframe

,ticker,stock_price,market_cap,number_of_shares_to_buy
0,AOS,85.160004,12420160512,23
1,ABT,106.230003,184800903168,18
2,MMM,126.760002,69636112384,15
3,ACN,331.130005,207414542336,6
4,ABBV,187.234207,330631249920,10
...,...,...,...,...
496,ZBRA,357.075012,18414358528,5
497,XYL,134.505005,34577068032,14
498,YUM,133.020004,37462691840,15
499,ZTS,181.729996,82922487808,10


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [20]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index=False) 

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [21]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)



### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [22]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.save()

# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Stock Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Cap', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', string_format)

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [23]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['Market Cap', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output

Saving our Excel file is very easy:

In [24]:
writer.close()